In [15]:
import numpy as np

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Conv1DClassifier(nn.Module):
    def __init__(self, num_classes):
        super(Conv1DClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.pool5 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv6 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=2)
        self.pool6 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Calculate the size of the feature map after all convolution and pooling layers
        # Input size is 65000
        final_length = 65000 // (2**6)  # After six max-pooling layers, each reducing size by half

        self.fc1 = nn.Linear(512 * final_length, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))
        x = self.pool4(self.relu(self.conv4(x)))
        x = self.pool5(self.relu(self.conv5(x)))
        x = self.pool6(self.relu(self.conv6(x)))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Generate a synthetic dataset (for demonstration purposes)
def generate_synthetic_data(num_samples, sequence_length, num_classes):
    data = np.load('ecgs_labels.npy')
    X = torch.FloatTensor(data[:,:-1]).unsqueeze(1)
    y = torch.LongTensor(data[:,-1])
    # X = torch.randn(num_samples, 1, sequence_length)
    # y = torch.randint(0, num_classes, (num_samples,))
    return X, y

# Parameters
num_classes = 3
sequence_length = 100  # Length of each input sequence
num_samples = 1000
batch_size = 32
num_epochs = 20
learning_rate = 0.001

# Generate data
X, y = generate_synthetic_data(num_samples, sequence_length, num_classes)
dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# print(X.shape)
# print(X.dtype)
# Initialize the model, loss function, and optimizer
model = Conv1DClassifier(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # print(inputs.shape)
        # print(labels.shape)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluation (for simplicity, using the same synthetic dataset)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


Epoch [1/20], Loss: 1.4170
Epoch [2/20], Loss: 0.3640
Epoch [3/20], Loss: 0.2489


KeyboardInterrupt: 